In [1]:
import jax
import jax.numpy as jnp

In [2]:
from LIMxCMBL.init import *


In [3]:
# Lambda_idx = int(sys.argv[1])
# nbins = int(sys.argv[2])
# ell_idx = int(sys.argv[3])

Lambda_idx = 18
nbins = 100
ell_idx = 0

Lambda = Lambdas[Lambda_idx]

# CCAT-prime
zmin = 3.5
zmax = 8.1

oup_fname = '/scratch/users/delon/LIMxCMBL/I_auto/comb'
oup_fname += '_zmin_%.1f_zmax_%.1f_Lambda_idx_%d_nb_%d_l_%d_tplquad.npy'%(zmin, zmax, Lambda_idx, nbins,ell_idx)
print(oup_fname)

/scratch/users/delon/LIMxCMBL/I_auto/comb_zmin_3.5_zmax_8.1_Lambda_idx_18_nb_100_l_0_tplquad.npy


In [4]:
Omega_field = 8 * (np.pi/180)**2 #rad^2
Pei = 2.3e4 #Mpc^3 kJy^2 /sr^2 
chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))

In [5]:
from scipy.interpolate import interp1d, LinearNDInterpolator
from scipy.integrate import quad, quad_vec, trapezoid, tplquad

inner_dkparp_integral = np.load('/oak/stanford/orgs/kipac/users/delon/LIMxCMBL/inner_dkparp_integral.npy')
tmp_chibs = []
tmp_log_deltas = []
tmp_fnctn = []
for i in range(len(chibs)):
    for j in range(len(deltas)):
        tmp_chibs += [chibs[i]]
        tmp_log_deltas += [np.log10(deltas[j])]
        tmp_fnctn += [inner_dkparp_integral[ell_idx,i,j]]
        
f_inner_integral = LinearNDInterpolator(list(zip(tmp_chibs, tmp_log_deltas)), tmp_fnctn)
f_inner_integral_LoLo = interp1d(x = chibs, y = inner_dkparp_integral[ell_idx], axis = 0)

In [7]:
from interpax import interp2d, interp1d
inner_dkparp_integral = jnp.array(inner_dkparp_integral.astype(np.float64))

In [8]:
f_inner_integral((7838, np.log10(0.001)))

array(752.78944199)

In [9]:
interp2d(xq = [7838], yq=[np.log(0.001)], 
         x = chibs, y = np.log(deltas), f=inner_dkparp_integral[ell_idx],
        method='linear',)

Array([752.7989], dtype=float32)

In [10]:
chi_bin_edges = jnp.linspace(chimin*(1+1e-8), chimax*(1 - 1e-8), nbins + 1)
chi_bin_centers = (chi_bin_edges[1:] + chi_bin_edges[:-1])/2
dchi_binned = jnp.mean(jnp.diff(chi_bin_edges))

In [11]:
from jax import jit

In [36]:
_tmp = np.load('LIMxCMBL/_KI.npy')
_chis, _KI = _tmp[0], _tmp[1]

In [40]:
_chis = jnp.array(_chis)
_KI = jnp.array(_KI)

In [46]:

def f_KI1D(chi):
    return np.interp(x = chi, xp = _chis, fp = _KI, left = 0, right = 0)



In [60]:
@jit
def f_KILo(chi, external_chi, Lambda):
    return (Lambda / jnp.pi 
            * jnp.interp(x = chi, xp = _chis, 
                         fp = _KI, left = 0, right = 0) 
            * jnp.sinc(Lambda * (external_chi - chi) / np.pi))

@jit
def f_integrand(_chib, chip, chi):
    # by assumption chimin <= chi,chip <= chimax
    #I Lo
    _delta = jnp.abs(1 - chi/_chib) #(1)
    _delta = jnp.where(_delta < 1e-6, 1e-6, _delta)
    _delta = jnp.where(_delta > 0.7, 0.7, _delta)
    c1    = jnp.where(chimin <= (2*_chib - chi), 
            jnp.where((2*_chib - chi) <= chimax, 
            jnp.where(chimin <= chi, 
            jnp.where(chi <= chimax,
                2 * jnp.interp(x = chi, xp = _chis, fp = _KI, left = 0, right = 0)
                * interp2d(xq = [_chib], yq=[jnp.log(_delta)], 
                           x = chibs, y = jnp.log(deltas), f=inner_dkparp_integral[ell_idx],
                           method='linear',)
                / (_chib**2)
                * f_KILo(2*_chib - chi, external_chi = chip, Lambda=Lambda),
                      0),0),0),0)
    _delta = jnp.abs(1 - chip/_chib) #(1)
    _delta = jnp.where(_delta < 1e-6, 1e-6, _delta)
    _delta = jnp.where(_delta > 0.7, 0.7, _delta)

    c2    = jnp.where(chimin <= (2*_chib - chip), 
            jnp.where((2*_chib - chip) <= chimax, 
            jnp.where(chimin <= chip, 
            jnp.where(chip <= chimax,
                2 * jnp.interp(x = chip, xp = _chis, fp = _KI, left = 0, right = 0)
                * interp2d(xq = [_chib], yq=[jnp.log(_delta)], 
                           x = chibs, y = jnp.log(deltas), f=inner_dkparp_integral[ell_idx],
                           method='linear',)
                / (_chib**2)
                * f_KILo(2*_chib - chip, external_chi = chi, Lambda=Lambda),
                      0),0),0),0)

    cross_integrand = c1+c2
    
    #LoLo
    plus = _chib*(1+deltas) #(d)
    mins = _chib*(1-deltas) #(d)
    _idxs = (chimin < plus) & (plus < chimax) & (chimin < mins) & (mins < chimax)
    LoLo_integrand  = jnp.where(_idxs,
                               f_KILo(plus, external_chi = chi, Lambda=Lambda) * f_KILo(mins, external_chi = chip, Lambda=Lambda),
                               0) #(d)
    LoLo_integrand += jnp.where(_idxs,
                               f_KILo(mins, external_chi = chi, Lambda=Lambda) * f_KILo(plus, external_chi = chip, Lambda=Lambda),
                               0) #(d)
    
    LoLo_integrand *= (2 / _chib) #(d)
    LoLo_integrand *= deltas
    LoLo_integrand *= interp1d(xq = [_chib],
                           x = chibs, f=inner_dkparp_integral[ell_idx],
                           method='linear',)
    LoLo_integrand = jnp.trapezoid(x = jnp.log(deltas), y = LoLo_integrand, axis=-1)
    return LoLo_integrand - cross_integrand

In [61]:
f_integrand(8000, 8001, 8004)

Array([-6.635824e-09], dtype=float32)

In [62]:
def r():
    return np.random.uniform(chimin, chimax)
_tmp = jnp.array([ (r(), r(), r()) for _ in range(10000)])

In [64]:
import cProfile
cProfile.run('[f_integrand(a,b,c).block_until_ready() for a,b,c in _tmp]')

         235882 function calls in 0.488 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.303    0.303    0.476    0.476 <string>:1(<listcomp>)
        1    0.012    0.012    0.488    0.488 <string>:1(<module>)
       32    0.000    0.000    0.000    0.000 __init__.py:96(_xla_gc_callback)
    20406    0.004    0.000    0.004    0.000 array.py:225(shape)
        1    0.000    0.000    0.000    0.000 array.py:229(dtype)
    10201    0.004    0.000    0.008    0.000 array.py:233(ndim)
        2    0.000    0.000    0.000    0.000 array.py:237(size)
    20004    0.003    0.000    0.003    0.000 array.py:241(sharding)
        1    0.000    0.000    0.000    0.000 array.py:258(__bool__)
        1    0.000    0.000    0.000    0.000 array.py:282(__index__)
    10001    0.013    0.000    0.041    0.000 array.py:338(__iter__)
    50001    0.099    0.000    0.131    0.000 array.py:344(<genexpr>)
    10003    0.005    0.0

In [65]:
from quadax import quadgk

In [72]:
def chib_quad(y, x, L, R):
    return quadgk(f_integrand, [L, R],
                  epsabs = 0.0, epsrel = 1e-3,
                  args=(y, x))[0]

def bin_j_quad(x, lj, L, rj, R):
    return quadgk(chib_quad, [lj, rj],
                  epsabs = 0.0, epsrel = 1e-3,
                  args=(x, L, R))[0]

def bin_i_quad(li, lj, L, ri, rj, R):
    return quadgk(bin_j_quad, [li, ri],
                  epsabs = 0.0, epsrel = 1e-3,
                  args=(lj, L, rj, R))[0]

In [73]:
i = 13
li, ri = chi_bin_edges[i], chi_bin_edges[i+1]

j = 14
lj, rj = chi_bin_edges[j], chi_bin_edges[j+1]


In [ ]:
bin_i_quad(li, lj, 10, ri, rj, chimax_sample)

/home/users/delon/.local/lib/python3.9/site-packages/jax/_src/core.py:678: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(
/home/users/delon/.local/lib/python3.9/site-packages/jax/_src/core.py:678: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(
/home/users/delon/.local/lib/python3.9/site-packages/jax/_src/core.py:678: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(
/home/users/delon/.local/lib/python3.9/site-packages/jax/_src/core.py:678: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lea